<a href="https://colab.research.google.com/github/MK316/Myapps/blob/main/TCE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TCE exam app code (24.11.25) before revision

In [ ]:
import gradio as gr
import pandas as pd

# Load the DataFrame
url = "TExam_new20241125.csv"
df = pd.read_csv(url, encoding='utf-8-sig')

# Function to search years based on the selected mode
def search_years(search_mode, query):
    if search_mode == "Search questions by YEAR":
        matches = df[df['YEAR'].str.startswith(query.strip()[:4])]  # Ensure query is stripped of whitespace
    elif search_mode == "Search questions by Keywords":
        keyword_list = [keyword.strip().lower() for keyword in query.split(',')]
        matches = df[df['KEYWORDS'].str.lower().apply(lambda x: any(keyword in x for keyword in keyword_list))]
    elif search_mode == "Search questions by Words":
        word_list = [word.strip().lower() for word in query.split(',')]
        matches = df[df['TEXT'].str.lower().apply(lambda x: any(word in x for word in word_list))]
    else:
        return [], "Please select a valid search mode."

    if matches.empty:
        return [], "No results found for your query."
    return matches['YEAR'].tolist(), "Search completed successfully."

# Function to get image HTML
def get_image_html(year):
    match = df[df['YEAR'] == year]
    if not match.empty:
        image_filename = match.iloc[0]['Filename']
        image_url = f'https://huggingface.co/spaces/MK-316/TCE/raw/main/TExams/{image_filename}'
        keywords = match.iloc[0]['TEXT']
        return f"<b>🌷 Keywords:</b> 🔑 {keywords}<br><img src='{image_url}' width='800'/>"
    else:
        return "No keywords found for this year.", "No image found for this year."

# Create Gradio Blocks interface
with gr.Blocks() as app:
    gr.Markdown("# Teacher Certificate Exam Searching Engine")
    gr.Markdown("## ❄️ [1] Search Data")

    # Radio buttons to select search mode
    search_mode = gr.Radio(choices=["Search questions by YEAR", "Search questions by Keywords", "Search questions by Words"], label="Search Mode")

    # Row for search query and button
    search_input = gr.Textbox(label="Search Query: e.g., 2024 (by YEAR) or tapping (by Keywords or Words)", placeholder="Enter year or keywords or words")
    search_button = gr.Button("Click to Search")
    search_output = gr.Dropdown(label="Results (file names)", choices=[], visible=False)
    status_output = gr.Textbox(label="Status", visible=False)

    # Connect actions
    search_button.click(fn=search_years, inputs=[search_mode, search_input], outputs=[search_output, status_output])

    gr.Markdown("## ❄️ [2] Display a Specific Exam Question Requested")
    year_input = gr.Dropdown(label="Select a file name from the results", choices=[])
    submit_button = gr.Button("Show me the exam question")
    image_output = gr.HTML()

    # Update dropdown and image display
    search_output.change(fn=lambda results: gr.update(choices=results), inputs=search_output, outputs=year_input)
    submit_button.click(fn=get_image_html, inputs=year_input, outputs=image_output)

# Launch the app with sharing options
app.launch(debug=True)